In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,atuona,PF,-9.8000,-139.0333,77.45,80,24,15.73,light rain,0.42,0.0
1,1,hithadhoo,MV,-0.6000,73.0833,84.42,76,69,18.90,broken clouds,0.00,0.0
2,2,punta arenas,CL,-53.1500,-70.9167,39.11,80,34,27.02,scattered clouds,0.00,0.0
3,3,barrow,US,71.2906,-156.7887,19.42,86,75,9.22,broken clouds,0.00,0.0
4,4,rikitea,PF,-23.1203,-134.9692,76.77,79,28,19.51,scattered clouds,0.00,0.0


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('What is the minimum temperature you would like for your trip?'))
max_temp = float(input('What is the maximum temperature you would like for your trip?'))

What is the minimum temperature you would like for your trip?20
What is the maximum temperature you would like for your trip?90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
rain_amount = input('Do you want it to be raining? (yes/no)')
snow_amount = input('Do you want it to be snowing? (yes/no)')
if rain_amount == 'no' and snow_amount == 'no':
    clean_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) &
                                          (city_data_df['Max Temp'] >= min_temp) &
                                          (city_data_df['Rain (inches)'] == 0) &
                                          (city_data_df['Snow (inches)'] == 0)]
    
elif rain_amount == 'no' and snow_amount == 'yes':
    clean_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) &
                                          (city_data_df['Max Temp'] >= min_temp) &
                                          (city_data_df['Rain (inches)'] == 0) &
                                          (city_data_df['Snow (inches)'] >= 0.0)]
elif rain_amount == 'yes' and snow_amount == 'no':
    clean_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) &
                                          (city_data_df['Max Temp'] >= min_temp) &
                                          (city_data_df['Rain (inches)'] > 0.0) &
                                          (city_data_df['Snow (inches)'] == 0)]
else:
    clean_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) &
                                          (city_data_df['Max Temp'] >= min_temp) &
                                          (city_data_df['Rain (inches)'] > 0.0) &
                                          (city_data_df['Snow (inches)'] > 0)]
clean_df.head()

Do you want it to be raining? (yes/no)no
Do you want it to be snowing? (yes/no)yes


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
1,1,hithadhoo,MV,-0.6000,73.0833,84.42,76,69,18.90,broken clouds,0.0,0.0
2,2,punta arenas,CL,-53.1500,-70.9167,39.11,80,34,27.02,scattered clouds,0.0,0.0
4,4,rikitea,PF,-23.1203,-134.9692,76.77,79,28,19.51,scattered clouds,0.0,0.0
5,5,petropavlovsk-kamchatskiy,RU,53.0452,158.6483,39.90,93,100,6.73,overcast clouds,0.0,0.0
6,6,tasiilaq,GL,65.6145,-37.6368,35.76,67,100,6.02,overcast clouds,0.0,0.0


In [5]:
# 4a. Determine if there are any empty rows.
clean_df.isnull().sum()

City_ID                0
City                   0
Country                4
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
Rain (inches)          0
Snow (inches)          0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = clean_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
1,1,hithadhoo,MV,-0.6000,73.0833,84.42,76,69,18.90,broken clouds,0.0,0.00
2,2,punta arenas,CL,-53.1500,-70.9167,39.11,80,34,27.02,scattered clouds,0.0,0.00
4,4,rikitea,PF,-23.1203,-134.9692,76.77,79,28,19.51,scattered clouds,0.0,0.00
5,5,petropavlovsk-kamchatskiy,RU,53.0452,158.6483,39.90,93,100,6.73,overcast clouds,0.0,0.00
6,6,tasiilaq,GL,65.6145,-37.6368,35.76,67,100,6.02,overcast clouds,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
554,554,kigonsera,TZ,-10.8000,35.0500,73.36,57,88,5.12,overcast clouds,0.0,0.00
555,555,dombarovskiy,RU,50.7587,59.5386,60.53,28,13,18.32,few clouds,0.0,0.00
556,556,jipijapa,EC,-1.3333,-80.5833,71.10,91,100,3.47,overcast clouds,0.0,0.00
557,557,bilibino,RU,68.0546,166.4372,24.13,97,100,2.66,light snow,0.0,0.21


In [7]:
clean_df.count()

City_ID                459
City                   459
Country                459
Lat                    459
Lng                    459
Max Temp               459
Humidity               459
Cloudiness             459
Wind Speed             459
Current Description    459
Rain (inches)          459
Snow (inches)          459
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = " "
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,hithadhoo,MV,84.42,broken clouds,-0.6000,73.0833,
2,punta arenas,CL,39.11,scattered clouds,-53.1500,-70.9167,
4,rikitea,PF,76.77,scattered clouds,-23.1203,-134.9692,
5,petropavlovsk-kamchatskiy,RU,39.90,overcast clouds,53.0452,158.6483,
6,tasiilaq,GL,35.76,overcast clouds,65.6145,-37.6368,
7,saint george,US,65.25,clear sky,37.1041,-113.5841,
8,hobart,AU,45.01,clear sky,-42.8794,147.3294,
9,vaini,TO,73.56,few clouds,-21.2000,-175.2000,
10,chicama,PE,67.77,overcast clouds,-7.8447,-79.1469,
11,port elizabeth,ZA,68.09,scattered clouds,-33.9180,25.5701,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Cannot find hotel...skipping.")  

Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot fin

In [10]:
# 7. Drop the rows where there is no Hotel Name.
c = hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df = c.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,hithadhoo,MV,84.42,broken clouds,-0.6000,73.0833,Scoop Guest House
2,punta arenas,CL,39.11,scattered clouds,-53.1500,-70.9167,Hotel Hain
4,rikitea,PF,76.77,scattered clouds,-23.1203,-134.9692,People ThankYou
5,petropavlovsk-kamchatskiy,RU,39.90,overcast clouds,53.0452,158.6483,The Hotel Petropavlovsky
6,tasiilaq,GL,35.76,overcast clouds,65.6145,-37.6368,The Red House
...,...,...,...,...,...,...,...
552,kaohsiung,TW,82.35,scattered clouds,22.6163,120.3133,御宿商旅-中山館
553,stykkisholmur,IS,45.18,overcast clouds,65.0757,-22.7297,Fosshótel Stykkishólmur
556,jipijapa,EC,71.10,overcast clouds,-1.3333,-80.5833,Hotel Los Ceibos Jipijapa
557,bilibino,RU,24.13,light snow,68.0546,166.4372,Gostinitsa Severyanka


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "vacation_search/weatherpy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))